In [1]:
import sys

sys.path.append('../')

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from probabilistic_covshift.constants.automl_constants import AutoMLConfig
from probabilistic_covshift.constants.automl_constants import H2OServerInfo
from probabilistic_covshift.constants.main_constants import OriginFeatures
from probabilistic_covshift.constants.main_constants import WeightFeatures
from probabilistic_covshift.probabilistic_classification_covshift import ProbabilisticClassification

In [2]:
spark = SparkSession.builder.appName('main').master('local[4]').getOrCreate()

In [3]:
source_df = spark.createDataFrame([
    (38.9, 40.0, 55, 10.0), (88.9, 50.0, 15, 20.0),
    (38.9, 50.0, 15, 10.0), (48.9, 40.0, 55, 20.0),
    (38.9, 40.0, 55, 10.0), (98.9, 50.0, 15, 20.0),
    (88.9, 50.0, 15, 20.0), (18.9, 40.0, 55, 30.0),
    (48.9, 40.0, 55, 20.0), (58.9, 50.0, 15, 30.0),
    (98.9, 50.0, 15, 20.0), (38.9, 40.0, 55, 10.0),
    (18.9, 40.0, 55, 30.0), (38.9, 50.0, 15, 10.0),
    (58.9, 50.0, 15, 30.0), (38.9, 40.0, 55, 10.0),
    (38.9, 40.0, 55, 10.0), (88.9, 50.0, 15, 20.0),
    (38.9, 50.0, 15, 10.0), (48.9, 40.0, 55, 20.0),
    (38.9, 40.0, 55, 10.0), (98.9, 50.0, 15, 20.0),
    (88.9, 50.0, 15, 20.0), (18.9, 40.0, 55, 30.0),
    (48.9, 40.0, 55, 20.0), (58.9, 50.0, 15, 30.0),
    (98.9, 50.0, 15, 20.0), (38.9, 40.0, 55, 10.0),
    (18.9, 40.0, 55, 30.0), (38.9, 50.0, 15, 10.0),
    (58.9, 50.0, 15, 30.0), (38.9, 40.0, 55, 10.0)],
    ['col_a', 'col_b', 'col_c', 'col_d'])

In [4]:
target_df = spark.createDataFrame([
    (18.9, 40.0, 95, 10.0), (38.9, 50.0, 15, 20.0),
    (18.9, 50.0, 95, 10.0), (38.9, 40.0, 55, 20.0),
    (18.9, 40.0, 95, 10.0), (38.9, 50.0, 15, 20.0),
    (18.9, 50.0, 95, 30.0), (38.9, 40.0, 55, 30.0),
    (18.9, 40.0, 95, 30.0), (38.9, 50.0, 15, 30.0),
    (38.9, 50.0, 95, 30.0), (18.9, 40.0, 55, 10.0),
    (38.9, 40.0, 95, 30.0), (18.9, 50.0, 15, 10.0),
    (38.9, 50.0, 95, 30.0), (18.9, 40.0, 55, 10.0),
    (38.9, 40.0, 55, 30.0), (58.9, 50.0, 15, 20.0),
    (38.9, 50.0, 15, 30.0), (58.9, 40.0, 55, 20.0),
    (38.9, 40.0, 55, 30.0), (58.9, 50.0, 15, 20.0),
    (58.9, 50.0, 15, 30.0), (58.9, 40.0, 55, 30.0),
    (58.9, 40.0, 55, 30.0), (58.9, 50.0, 15, 30.0),
    (58.9, 50.0, 15, 30.0), (58.9, 40.0, 55, 10.0),
    (58.9, 40.0, 55, 30.0), (58.9, 50.0, 15, 10.0),
    (58.9, 50.0, 15, 30.0), (58.9, 40.0, 55, 10.0)],
    ['col_a', 'col_b', 'col_c', 'col_d'])

# Compute weights

In [5]:
conf = {
    AutoMLConfig.DATA: {
        AutoMLConfig.LABEL_COL: OriginFeatures.ORIGIN,
        AutoMLConfig.WEIGHT_COL: WeightFeatures.WEIGHT,
        AutoMLConfig.BASE_TABLE_PATH: 'data/base_table.parquet',
        AutoMLConfig.WEIGHT_PATH: 'data/weight.csv'
    },
    AutoMLConfig.SERVER_CONN_INFO: {
        H2OServerInfo.IP: 'localhost',
        H2OServerInfo.PORT: '54321'
    },
    AutoMLConfig.CROSS_VAL: {
        AutoMLConfig.FOLD_COL: "fold",
        AutoMLConfig.NFOLDS: 8,
    },
    AutoMLConfig.MODELING: {
        AutoMLConfig.MAX_RUNTIME_SECS: 3600,
        AutoMLConfig.MAX_MODELS: 10,
        AutoMLConfig.STOPPING_METRIC: 'logloss',
        AutoMLConfig.SORT_METRIC: 'logloss'
    },
    AutoMLConfig.EXCLUDE_ALGOS: [
        "StackedEnsemble",
        "DeepLearning"
    ],
    AutoMLConfig.MODEL: {
        AutoMLConfig.BEST_MODEL_PATH: 'data/model/'
    },
    AutoMLConfig.SEED: 23
}

In [6]:
pc = ProbabilisticClassification(source_df, target_df, conf)
pc.run()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_191"; Java(TM) SE Runtime Environment (build 1.8.0_191-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.191-b12, mixed mode)
  Starting server from /Users/albertus.kelvin/Documents/GLAIR_OWN/invstr-credit-scoring/env/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/c4/drbdtk0d67b4g99gp19b655c0000gp/T/tmpb4xrfjav
  JVM stdout: /var/folders/c4/drbdtk0d67b4g99gp19b655c0000gp/T/tmpb4xrfjav/h2o_albertus_kelvin_started_from_python.out
  JVM stderr: /var/folders/c4/drbdtk0d67b4g99gp19b655c0000gp/T/tmpb4xrfjav/h2o_albertus_kelvin_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,07 secs
H2O cluster timezone:,Asia/Jakarta
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.2
H2O cluster version age:,2 months and 5 days
H2O cluster name:,H2O_from_python_albertus_kelvin_l5wysn
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,0
H2O cluster allowed cores:,0
H2O cluster status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Base table inferred column types: {'col_a': 'real', 'col_b': 'int', 'col_c': 'int', 'col_d': 'int', 'origin': 'enum', 'row_id': 'int'}


INFO:probabilistic_covshift.automl.trainer:Base table inferred column types: {'col_a': 'real', 'col_b': 'int', 'col_c': 'int', 'col_d': 'int', 'origin': 'enum', 'row_id': 'int'}


AutoML progress: |
20:50:13.449: Fold column fold will be used for cross-validation. nfolds parameter will be ignored.

███████████████
20:50:24.691: Skipping training of model GBM_5_AutoML_20200325_205013 due to exception: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_5_AutoML_20200325_205013.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 64.0.


█████████████████████████████████████████| 100%


...


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,95.0,95.0,20622.0,0.0,6.0,5.189474,1.0,19.0,12.642105


Loaded model: 


INFO:probabilistic_covshift.automl.predictor:Loaded model: 


gbm prediction progress: |████████████████████████████████████████████████| 100%
Export File progress: |███████████████████████████████████████████████████| 100%


# Append the weight to the base table

In [7]:
base_frame_df = spark.read.parquet(conf[AutoMLConfig.DATA][AutoMLConfig.BASE_TABLE_PATH])\
                     .drop(conf[AutoMLConfig.DATA][AutoMLConfig.LABEL_COL])
base_frame_df.toPandas().head()

,col_a,col_b,col_c,col_d,row_id
0,38.9,40.0,55,10.0,1
1,88.9,50.0,15,20.0,2
2,38.9,50.0,15,10.0,3
3,48.9,40.0,55,20.0,4
4,38.9,40.0,55,10.0,5


In [8]:
weight_df = spark.read.csv(conf[AutoMLConfig.DATA][AutoMLConfig.WEIGHT_PATH], header=True)
weight_df.toPandas().head()

,row_id,weight
0,1,0.0036418907088322564
1,2,8.131772727181886E-4
2,3,0.0038943427410750545
3,4,9.556472283483082E-4
4,5,0.0036418907088322564


In [9]:
weighted_base_frame_df = base_frame_df.join(weight_df, how='left', on='row_id')
weighted_base_frame_df.toPandas().head()

,row_id,col_a,col_b,col_c,col_d,weight
0,1,38.9,40.0,55,10.0,0.0036418907088322564
1,2,88.9,50.0,15,20.0,8.131772727181886E-4
2,3,38.9,50.0,15,10.0,0.0038943427410750545
3,4,48.9,40.0,55,20.0,9.556472283483082E-4
4,5,38.9,40.0,55,10.0,0.0036418907088322564
